# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.114074e+02     1.488169e+00
 * time: 0.38480091094970703
     1     1.037164e+01     9.242486e-01
 * time: 2.3753130435943604
     2    -1.155768e+01     1.025110e+00
 * time: 3.065113067626953
     3    -3.363660e+01     8.410566e-01
 * time: 4.09608006477356
     4    -4.679595e+01     7.364216e-01
 * time: 5.1269690990448
     5    -5.669423e+01     2.660630e-01
 * time: 6.161010026931763
     6    -5.955657e+01     2.716569e-01
 * time: 6.849260091781616
     7    -6.075238e+01     1.154856e-01
 * time: 7.53971004486084
     8    -6.128103e+01     4.055306e-02
 * time: 8.234080076217651
     9    -6.154368e+01     3.336302e-02
 * time: 8.925647020339966
    10    -6.172203e+01     3.140554e-02
 * time: 9.608704090118408
    11    -6.186452e+01     2.172721e-02
 * time: 10.301640033721924
    12    -6.196431e+01     1.914119e-02
 * time: 11.002727031707764
    13    -6.202419e+01     2.068268e-02
 * time: 11.704447031021118
    1

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671073
    AtomicLocal         -18.8557687
    AtomicNonlocal      14.8522646
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485377 
    Xc                  -19.3336822

    total               -62.261666462429
